In [2]:
# Import module
import os
import open_clip
import glob
import torch
import numpy as np
from PIL import Image
from tqdm import tqdm
import faiss


In [3]:
import faiss

# Path to your binary file
index_file_path = "./../dict/faiss_clipv2_cosine.bin"

# Load the index
index = faiss.read_index(index_file_path)

# Now you can use the index for search or further operations


In [14]:
##### Load Model #####
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model, _, preprocess = open_clip.create_model_and_transforms('ViT-L-14', device=device, pretrained='datacomp_xl_s13b_b90k')

cuda


In [44]:
os.getcwd()

'/home/nguyen/.code/AIChallenge2023/utils'

In [4]:
import numpy as np
import os

# Base directories
dataset_base_dir = './../data/keyframes/'
npy_base_dir = './../data_extraction/CLIP_features/'

# Initialize the mapping dictionary
frame_to_feature_map = {}

# Loop over each L0x directory
for level_dir in sorted(os.listdir(dataset_base_dir)):
    if level_dir.startswith('Keyframes_'):
        level_id = level_dir.split('_')[1]  # Extract L0x
        
        # Loop over each video directory within the L0x directory
        for video_dir in sorted(os.listdir(os.path.join(dataset_base_dir, level_dir))):
            video_id = video_dir.split('_')[1]  # Extract V00y

            # Load the features from the corresponding npy file
            npy_file_path = os.path.join(npy_base_dir, level_id, f'{video_id}.npy')
            if os.path.exists(npy_file_path):
                features = np.load(npy_file_path)
            else:
                print(f"Warning: Missing npy file for {level_id}_{video_id}")
                continue

            # Get the list of frames in the video directory
            frame_files = sorted(os.listdir(os.path.join(dataset_base_dir, level_dir, video_dir)))

            # Map each frame to its corresponding feature
            for i, frame_file in enumerate(frame_files):
                if frame_file.endswith('.jpg'):
                    frame_index = os.path.splitext(frame_file)[0]
                    key = f"{level_id}_{video_id}_{frame_index}"
                    frame_to_feature_map[key] = features[i].reshape(1,-1)

In [65]:
def search_tags(query_tags, model, index, embeddings, top_k=5):
    """Search for tags similar to the given query tags."""
    text_tokens = open_clip.tokenize(query_tags).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_tokens)

    text_features /= text_features.norm(dim=-1, keepdim=True)
    text_features = text_features.cpu().detach().numpy().astype(np.float32)
    
    distances, indices = index.search(text_features, top_k)

    scores, idx_image = index.search(text_features, k=top_k)

    idx_image = idx_image.flatten()
    
    results = []
    for i in indices[0]:
        key = list(embeddings.keys())[i]
        results.append(key)
    
    return results

In [71]:
# Example query
query = ["A man with g jacket"]
results = search_tags(query, model, index, frame_to_feature_map)

In [70]:
vid = "L0_V026_117"

NameError: name 'L08_V001_139' is not defined

In [69]:
print(results.index(vid))

0


In [72]:
print(results)

['L11_V018_222', 'L07_V025_186', 'L11_V018_219', 'L07_V011_267', 'L06_V001_193']


In [5]:
vector = index.reconstruct(0)

In [11]:
scores, idx_image = index.search(vector, k=5)

ValueError: not enough values to unpack (expected 2, got 1)

In [9]:
print(list(frame_to_feature_map.keys())[0])

L01_V001_005
